<a href="https://colab.research.google.com/github/Pushpit07/Spam_Email_Detection-Text_Processing-Naive_Bayes/blob/main/naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes Classifier

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('./mushrooms.csv')

In [ ]:
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,stalk_root,stalk_surface_above_ring,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


In [ ]:
df.shape

(8124, 23)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
le = LabelEncoder()

In [ ]:
ds = df.apply(func=le.fit_transform) # this will be applied to each column

In [ ]:
ds.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,stalk_root,stalk_surface_above_ring,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1


In [ ]:
data = ds.values
X = data[:, 1:]
y = data[:, 0]

In [ ]:
X.shape, y.shape

((8124, 22), (8124,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

# Custom NB

In [13]:
class CustomNB:
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    # label = which class you want this prob for
    def prior_prob(self, label): 
        total = self.y_train.shape[0]
        class_examples = np.sum(self.y_train == label)
        return class_examples / float(total)  # python 2
    
    # P(Xi=red|y=label) - ith feature (feature col = i) for a single example
    def conditional_prob(self, feature_col, feature_val, label):
        # out of all the examples, what mushrooms have feature as feature_val in the feature_col that belongs to that class label
        X_filtered = self.X_train[self.y_train==label] # all the examples in class label
        numerator = np.sum(X_filtered[:, feature_col] == feature_val)
        denominator = len(X_filtered)
        return numerator / denominator
    
    # we are going to do this for all the 22 features that we have for each example
    def predict_point(self, X_test):
        # X_test is a single example with n features
        classes = np.unique(self.y_train) # By default from 0
        n_features = self.X_train.shape[1]
        post_pro = []
        # post prob for each class
        for label in classes:
            # post_prob = prior * likelihood
            likehood = 1.0
            for feature in range(n_features):
                cond = self.conditional_prob(feature, X_test[feature], label)
                likehood *= cond
            prior = self.prior_prob(label)
            post = prior * likehood
            post_pro.append(post)
        
        # ans = max value from all labels
        return np.argmax(post_pro) # return the index of the largest value in array
    
    def predict(self, X_test):
        result = []
        for point in X_test:
            result.append(self.predict_point(point))
        return np.array(result)
    
    def score(self, X_test, y_test):
        return (self.predict(X_test) == y_test).mean()

In [14]:
model = CustomNB()

In [15]:
model.fit(X_train, y_train)

In [16]:
model.predict(X_test[:10])

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0])

In [17]:
y_test

array([0, 1, 1, ..., 0, 0, 0])

In [18]:
y_test[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0])

In [19]:
model.score(X_test, y_test)

0.9973890339425587